# Setup Minedojo Environment

In [1]:
from Environments import SkyRunner, MultithreadGym
frame_stack = True
frames = 4
use_grayscale=True

# Multithreaded environment wrapper
env = MultithreadGym.MultithreadGym(thread_int=1, env_int=1, frame_stack=frame_stack, frames_int=frames, use_grayscale=use_grayscale)

# Evaluation environment
eval_env = SkyRunner.CustomEnv(frame_stack=frame_stack, frames_int=frames, use_grayscale=use_grayscale)

[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


starting Reloader 0
ThreadID 0 waiting for environment to reset.
ThreadID: 0 has received an enviornment from queue. Reset of envornment is being prepeared


# Begin training

In [ ]:
import train_openAI
import importlib

importlib.reload(train_openAI)

train_openAI.train(env, eval_env=eval_env, eval_freq=5000)

Waiting for ready enviornment


/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/venv/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:229: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 14.41GB > 7.49GB
  warnings.warn(
[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


ThreadID 0 put complete environment into ready-queue.Ready enviornment received

ThreadID 0 waiting for environment to reset.
Inventory and weather cleared!
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Inventory and weather cleared!
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Inventory and weather cleared!
Broken block detected. Moving to location 1
Inventory and weather cleared!
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 wa

# Shutdown Environments

In [ ]:
env.close()
eval_env.close()

# Load existing model

In [4]:
from stable_baselines3 import DQN
from CustomBaselines3.DoubleDQN import DoubleDQN

_env = eval_env
model = DQN.load("last_model.zip")

obs = env.reset()
acc_r = 0
while True:
    act, st = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(act)

    acc_r += reward

    env.render()

    if done:
        obs = env.reset()
        print("Finished with reward %d" % acc_r)
        acc_r = 0

Inventory and weather cleared!
Finished with reward 0
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Inventory and weather cleared!
Finished with reward 0
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying aga

Failed to take a step. Error msg: a bytes-like object is required, not 'NoneType'


Unable to reset enviornment due to an exception
Traceback (most recent call last):
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/Environments/SkyRunner.py", line 83, in reset
    local_obs = self.env.reset()
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/Environments/Skyrunner_mission_interpreter.py", line 140, in reset
    self.quick_reset()
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/Environments/Skyrunner_mission_interpreter.py", line 205, in quick_reset
    self.env.kill_agent()
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/venv/lib/python3.9/site-packages/minedojo/sim/sim.py", line 569, in kill_agent
    return self._cmd_executor.kill_agent(action)
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/venv/lib/python3.9/site-packages/minedojo/sim/cmd_executor.py", line 139, in kill_agent
    obs, _, _, info = self.execute_cmd("/kill", action)
  File "/Users/kristian.aars/PycharmProjects/ID

Attempted to send kill command to minecraft process and failed with exception [Errno 61] Connection refused


Finished with reward 4


ValueError: Error: Unexpected observation shape () for Box environment, please use (4, 1, 60, 60) or (n_env, 4, 1, 60, 60) for the observation shape.